In [158]:
# using SQLalchemy 
from sqlalchemy import create_engine
import pandas as pd
import pyodbc
import pandas as pd
import numpy as np

database = 'hackathon_danone'
username = 'hack12'
password = 'boA8AKUb54!'
driver= '{SQL Server}'
server = '192.168.250.3'

my_engine = create_engine(f"""mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver={server}', fast_executemany=True""" )

def db_sql(sql):
    cnxn = pyodbc.connect(f'DRIVER={driver};PORT=1433;SERVER={server};DATABASE={database};UID={username};PWD={password}')
    data = pd.read_sql(sql,cnxn)
    cnxn.close()    
    return data


In [159]:
import datetime

In [160]:
raw_mat_in_reg = db_sql('select * from eval_recipe_0_raw_material_in')
raw_mat_used_reg = db_sql('select * from eval_recipe_0_raw_material_used')

In [161]:
raw_bigbag_reg = db_sql('select * from eval_recipe_0_processing_details_bigbag')
raw_dd_reg = db_sql('select * from eval_recipe_0_processing_details_dd')
raw_slurry_reg = db_sql('select * from eval_recipe_0_processing_details_slurry')

In [162]:
raw_mat_reg = pd.read_csv('raw_weighted_recipe0.csv', sep=',')

In [168]:
raw_dd_reg.head()

,orders_details_id,testing_time,steam_preasure,dd_speed,temp_out
0,0,2018-04-04 11:30:00,7.5,9.0,116.0
1,0,2018-04-04 12:30:00,7.8,9.5,116.0
2,0,2018-04-04 13:30:00,7.8,9.5,116.0
3,0,2018-04-04 14:30:00,7.5,9.5,116.0
4,0,2018-04-04 15:30:00,7.5,9.0,116.0


In [169]:
raw_bigbag_reg = raw_bigbag_reg.dropna()

In [165]:
raw_out_reg.head()

,orders_details_id,line_dd,process_order,testing_time,moisture,bulk_density
0,0,DD02,101065632.0,2018-03-21 06:30:00,4.99,240.0
1,0,DD02,101065632.0,2018-03-21 07:30:00,4.99,240.0
2,0,DD02,101065632.0,2018-03-21 08:30:00,4.74,250.0
3,0,DD02,101065632.0,2018-03-21 09:30:00,4.68,250.0
4,0,DD02,101065632.0,2018-03-21 10:30:00,4.71,250.0


In [166]:
out_bb_reg = raw_out_reg.merge(raw_bigbag_reg, on=['orders_details_id', 'bigbag_number'])
out_bb_reg = out_bb.dropna()
out_bb.head()

KeyError: 'bigbag_number'

In [170]:
out_dd = pd.merge_asof(raw_bigbag_reg.sort_values('bigbag_filling_time_end'), raw_dd.sort_values('testing_time'),
                      left_on='bigbag_filling_time_end', right_on='testing_time')
out_dd.head(5)

,orders_details_id_x,bigbag_number,bigbag_filling_time_end,sifter_speed_nominal_pct,orders_details_id_y,testing_time,steam_preasure,dd_speed,temp_out
0,0,1,2018-04-04 13:00:00,0.69999999999999996,4,2018-04-04 12:30:00.000,7.0,9.0,120.0
1,0,2,2018-04-04 14:40:00,0.69999999999999996,5,2018-04-04 14:30:00.000,7.0,9.0,120.0
2,0,3,2018-04-04 15:50:00,0.69999999999999996,5,2018-04-04 15:30:00.000,7.0,9.0,120.0
3,0,4,2018-04-04 17:30:00,0.69999999999999996,5,2018-04-04 16:30:00.007,7.0,9.0,120.0
4,0,5,2018-04-04 19:10:00,0.69999999999999996,5,2018-04-04 18:30:00.017,7.0,9.0,120.0


In [171]:
out_sl = pd.merge_asof(out_dd.sort_values('testing_time'), raw_slurry.sort_values('slurry_start_time'),
                      left_on='testing_time', right_on='slurry_start_time')
out_sl['start'] = out_sl.apply(lambda x: x['bigbag_filling_time_end'] - datetime.timedelta(minutes=20), axis=1)

In [127]:
datas = []
for i in range(len(out_sl)):
    start = out_sl['start'][i].strftime('%Y-%m-%d %H:%M:%S')
    end = out_sl['bigbag_filling_time_end'][i].strftime('%Y-%m-%d %H:%M:%S')
    data = db_sql(f"""
    SELECT
        AVG(w.steam_pressure_at_the_inlet_of_regulation_unit),
        MIN(w.steam_pressure_at_the_inlet_of_regulation_unit),
        MAX(w.steam_pressure_at_the_inlet_of_regulation_unit),
        AVG(w.product_temperature_at_the_outlet_of_JetCooker),
        MIN(w.product_temperature_at_the_outlet_of_JetCooker),
        MAX(w.product_temperature_at_the_outlet_of_JetCooker),
        AVG(w.product_temperature_at_the_inlet),
        MIN(w.product_temperature_at_the_inlet),
        MAX(w.product_temperature_at_the_inlet),
        AVG(w.steam_pressure_at_the_inlet_of_JetCooker),
        MIN(w.steam_pressure_at_the_inlet_of_JetCooker),
        MAX(w.steam_pressure_at_the_inlet_of_JetCooker),
        AVG(w.condensate_temperature_at_DD_outlet),
        MIN(w.condensate_temperature_at_DD_outlet),
        MAX(w.condensate_temperature_at_DD_outlet)
    FROM
        "Walec DD08" as w
    WHERE
        timestamp between '{start}' and '{end}'""")
    datas.append(data)

In [128]:
out_sl = out_sl.iloc[:, 1:]

In [172]:
out = out_sl.merge(raw_mat, left_on='orders_details_id_x', right_on='id')

In [173]:
out.to_csv('recipe_0_val_reg.csv')

In [174]:
out

,orders_details_id_x,bigbag_number,bigbag_filling_time_end,sifter_speed_nominal_pct,orders_details_id_y,testing_time,steam_preasure,dd_speed,temp_out,orders_details_id,...,water_correction,start,Unnamed: 0,id,particles_grp1,particles_grp2,particles_grp3,moisture,usage_pct,elems
0,0,1,2018-04-04 13:00:00,0.69999999999999996,4,2018-04-04 12:30:00.000,7.0,9.0,120.0,4,...,-40.0,2018-04-04 12:40:00,0,0,0.789210,0.209790,0.0,11.8881,0.999,3
1,0,2,2018-04-04 14:40:00,0.69999999999999996,5,2018-04-04 14:30:00.000,7.0,9.0,120.0,5,...,-40.0,2018-04-04 14:20:00,0,0,0.789210,0.209790,0.0,11.8881,0.999,3
2,0,3,2018-04-04 15:50:00,0.69999999999999996,5,2018-04-04 15:30:00.000,7.0,9.0,120.0,5,...,-40.0,2018-04-04 15:30:00,0,0,0.789210,0.209790,0.0,11.8881,0.999,3
3,0,4,2018-04-04 17:30:00,0.69999999999999996,5,2018-04-04 16:30:00.007,7.0,9.0,120.0,5,...,-40.0,2018-04-04 17:10:00,0,0,0.789210,0.209790,0.0,11.8881,0.999,3
4,0,5,2018-04-04 19:10:00,0.69999999999999996,5,2018-04-04 18:30:00.017,7.0,9.0,120.0,5,...,-40.0,2018-04-04 18:50:00,0,0,0.789210,0.209790,0.0,11.8881,0.999,3
5,0,6,2018-04-04 20:00:00,0.69999999999999996,5,2018-04-04 19:30:00.020,7.0,9.0,120.0,5,...,-40.0,2018-04-04 19:40:00,0,0,0.789210,0.209790,0.0,11.8881,0.999,3
6,0,7,2018-04-04 21:10:00,0.69999999999999996,5,2018-04-04 20:30:00.027,7.0,9.0,120.0,5,...,-40.0,2018-04-04 20:50:00,0,0,0.789210,0.209790,0.0,11.8881,0.999,3
7,0,8,2018-04-04 22:25:00,0.69999999999999996,5,2018-04-04 21:30:00.030,7.0,9.0,120.0,5,...,-40.0,2018-04-04 22:05:00,0,0,0.789210,0.209790,0.0,11.8881,0.999,3
8,0,9,2018-04-04 23:35:00,0.69999999999999996,5,2018-04-04 23:30:00.040,7.0,9.0,120.0,5,...,-40.0,2018-04-04 23:15:00,0,0,0.789210,0.209790,0.0,11.8881,0.999,3
9,0,10,2018-04-05 00:45:00,0.69999999999999996,5,2018-04-05 00:30:00.047,7.0,9.0,120.0,5,...,-40.0,2018-04-05 00:25:00,0,0,0.789210,0.209790,0.0,11.8881,0.999,3
